In [2]:
%pip install thefuzz

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from thefuzz import fuzz
from thefuzz import process
import pandas as pd
import numpy as np
import re

In [4]:
labels = pd.read_csv('data_all\labels_all.csv')

In [5]:
labels.head()

,Movie,Kinopoisk,Level,Subtitles
0,Forrest Gump,Rus sub,"A2/A2+, B1",Yes
1,Finding Nemo\n,Everything,A2/A2+,Yes
2,Cast away\n,"Paid, Rus sub",A2/A2+,Yes
3,The invisible man (2020)\n,"Paid, Rus lan",A2/A2+,Yes
4,Back to the future\n,Rus sub,A2/A2+,Yes


In [6]:
labels.describe()

,Movie,Kinopoisk,Level,Subtitles
count,116,69,116,88
unique,115,10,6,2
top,Sleepless in Seattle,Everything,B1,Yes
freq,2,19,38,85


In [7]:
#приведем названия колонок к одному формату
labels = labels.rename(str.lower, axis='columns')
labels.head()

,movie,kinopoisk,level,subtitles
0,Forrest Gump,Rus sub,"A2/A2+, B1",Yes
1,Finding Nemo\n,Everything,A2/A2+,Yes
2,Cast away\n,"Paid, Rus sub",A2/A2+,Yes
3,The invisible man (2020)\n,"Paid, Rus lan",A2/A2+,Yes
4,Back to the future\n,Rus sub,A2/A2+,Yes


In [8]:
#обработаем названия фильмов 
labels.movie=labels['movie'].apply(lambda x: re.sub(r'(<.*?>)|([^\w\s])', '', x).replace('\n', '').replace(' ', '_').lower())
labels = labels.sort_values(by='movie')
labels = labels.reset_index(drop=True)
labels.head()

,movie,kinopoisk,level,subtitles
0,10_cloverfield_lane,NaN,B1,Yes
1,10_things_i_hate_about_you,No subs,B1,Yes
2,a_knights_tale,Everything,B2,Yes
3,a_star_is_born,Nope,B2,Yes
4,aladdin,Everything,A2/A2+,Yes


In [9]:
#приведем к одному виду все файлы субтитров
def to_lowcase(source):
    path = source
    files = os.listdir(path)
    i=0
    for file in files:
        filename, file_extension = os.path.splitext(file)
        os.rename(os.path.join(path, file), os.path.join(path, filename.lower() + file_extension))
        i+=1
    return f'{i} files renamed'

In [10]:
path = 'data_all\Subtitles_all'
to_lowcase(path)

'115 files renamed'

In [11]:
content = os.listdir(path) 

In [12]:
def search_eng(data, folder):
    result = []
    for i in range(len(data)):
        result.append(process.extractOne(data[i], folder)[0])
    return result

In [13]:
labels['subs']=pd.Series(search_eng(labels['movie'], content))

In [14]:
labels=labels.drop_duplicates(subset='movie')
labels.tail(30)

,movie,kinopoisk,level,subtitles,subs
85,the_graduate,Nope,"B1, B2",Yes,the_graduate(1967).srt
86,the_greatest_showman_,NaN,A2/A2+,Yes,the_greatest_showman(2017).srt
87,the_hangover,NaN,B2,Yes,the_hangover(2009).srt
88,the_holiday,No subs,B1,Yes,the_holiday(2006).srt
89,the_intern,NaN,B2,NaN,the_intern(2015).srt
90,the_invisible_man_2020,"Paid, Rus lan",A2/A2+,Yes,the_invisible_man(2020).srt
91,the_jungle_book,Everything,A2/A2+,Yes,the_jungle_book(2016).srt
92,the_kings_speech,NaN,B2,Yes,the_kings_speech(2010).srt
93,the_legend_of_tarzan,NaN,C1,NaN,the_legend_of_tarzan(2016).srt
94,the_lion_king,Everything,A2/A2+,Yes,the_lion_king(1994).srt


In [15]:
labels.to_csv('labels.csv')

Getting information, Please Wait....
'NoneType' object is not subscriptable
Invalid input or Network Error!


In [2]:
idioms = open('1234.txt'); 
folder = open('data_all\Subtitles_txt\my_big_fat_greek_wedding(2002).txt')
idioms.readlines()

['\n',
 'a bear with a sore head a bit of a dark horse\n',
 "a bitter pill to swallow a different kettle of fish a dog's life\n",
 'a fish out of water a long shot\n',
 'a memory like a sieve a night owl\n',
 'a pain in the neck a piece of cake\n',
 'a red rag to a bull a red-letter day\n',
 'a sitting duck\n',
 'a storm in a teacup a wild-goose chase above board Achilles heel against all odds agree to disagree \n',
 'all along\n',
 'all but all in  all in all\n',
 'all the same all told\n',
 "an old wives' tale\n",
 'an unknown quantity as a last resort\n',
 'as the crow flies at close quarters at large\n',
 "at the cutting edge bark up the wrong tree be ahead of one's time be all at sea\n",
 'be all fingers and thumbs be all very well\n',
 'be as busy as a bee be as thick as a brick\n',
 'be beside oneself with anger be born yesterday\n',
 'be broke\n',
 'be caught red-handed be dying for sth\n',
 'be fit for\n',
 'be for the high jump be full of beans\n',
 'be green\n',
 'be in a qu

In [10]:
def search_eng(data, folder):
    result = []
    for i in data:
        result.append(process.extractOne(data[i], folder)[1])
    return result

In [14]:
search_eng(idioms.readlines(), folder)

[]